In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
del spacex_df[spacex_df.columns[0]]


In [3]:
spacex_df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
site_names = list(spacex_df['Launch Site'].unique())
site_names

['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']

In [5]:
site_option_list = []
for i in site_names:
    site_option_list.append({'label':str(i), 'value':i})
site_option_list.append({'label':str('All Sites'), 'value':'All'})
site_option_list

[{'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
 {'label': 'All Sites', 'value': 'All'}]

In [6]:
# Create a dash application
app = JupyterDash(__name__)
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown', options=site_option_list, value ='All', placeholder = "All Sites", searchable = True,
                                             style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),
                                html.Br(),   
                                
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                   
                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min=0, max=10000, step=1000,
                                                value=[min_payload, max_payload]),
                                html.Br(),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),                          
                               
                               ])
                               
                        
        
                               
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    
    if entered_site == 'All':
        fig = px.pie(spacex_df.groupby('Launch Site').sum().reset_index(), values='class', 
                     names='Launch Site',title='Total Success Launches by site')
        return fig
    else:
        fig = px.pie(spacex_df[spacex_df['Launch Site']==entered_site].groupby('class').count().reset_index(), 
                     values = 'Launch Site', names='class', title = 'Total Success Launches for Site '+str(entered_site))
        return fig
    
#TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id="payload-slider", component_property="value")]
             )

def get_scatter_plot(entered_site, payload_value): 
    
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)']>payload_value[0]) & (spacex_df['Payload Mass (kg)']<payload_value[1])].reset_index()
    
    if entered_site == 'All':
        req_df = filtered_df
        fig = px.scatter(req_df, x= 'Payload Mass (kg)', y= 'class', color ='Booster Version Category',
                        title = 'Correlation Between Payload and Success for all sites')
        return fig
    else:
        req_df = filtered_df[filtered_df['Launch Site']==entered_site]
        fig = px.scatter(req_df, x= 'Payload Mass (kg)', y= 'class', color ='Booster Version Category',
                        title = 'Correlation Between Payload and Success for '+str(entered_site))
        return fig











In [7]:
# Run the app
if __name__ == '__main__':    
    app.run_server(mode="inline", host="localhost", debug=False, dev_tools_ui=False, dev_tools_props_check=False)

 * Running on http://localhost:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Jun/2022 22:24:29] "GET /_alive_87658dd8-6485-4bf1-835e-f82f886fe723 HTTP/1.1" 200 -


127.0.0.1 - - [22/Jun/2022 22:24:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2022 22:24:30] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2022 22:24:31] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2022 22:24:31] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2022 22:24:31] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2022 22:24:31] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2022 22:24:31] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2022 22:24:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2022 22:24:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2022 22:24:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2022 22:24:41] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2022 22:24:41] "GET /_dash-dependencies